# Machine Learning Foundation

## Section 2, Part e: Regularization LAB


## Learning objectives

By the end of this lesson, you will be able to:

*   Implement data standardization
*   Implement variants of regularized regression
*   Combine data standardization with the train-test split procedure
*   Implement regularization to prevent overfitting in regression problems


In [1]:
import piplite
await piplite.install(['tqdm', 'seaborn', 'pandas', 'numpy'])

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

np.set_printoptions(precision=3, suppress=True)

In the following cell we load the data and define some useful plotting functions.


In [7]:
np.random.seed(72018)

# from sklearn.datasets import load_boston

def to_2d(array):
    return array.reshape(array.shape[0], -1)


    
def plot_exponential_data():
    data = np.exp(np.random.normal(size=1000))
    plt.hist(data)
    plt.show()
    return data
    
def plot_square_normal_data():
    data = np.square(np.random.normal(loc=5, size=1000))
    plt.hist(data)
    plt.show()
    return data

### Loading in Boston Data


In [6]:
#This function will download the dataset into your browser 
 
from pyodide.http import pyfetch
 
async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())
 
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv"
 
#you will need to download the dataset; if you are running locally, please comment out the following 
await download(path, "boston_housing.csv")
 
 
# Import pandas library
import pandas as pd
 
# Read the online file by the URL provides above, and assign it to variable "df"
 
boston_data = pd.read_csv("boston_housing.csv")
 
# show the first 5 rows using dataframe.head() method
print("The first 5 rows of the dataframe") 
boston_data.head(5)

The first 5 rows of the dataframe


,Unnamed: 0,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,4.98,24.0
1,1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,9.14,21.6
2,2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,4.03,34.7
3,3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,2.94,33.4
4,4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,5.33,36.2


## Data standardization


**Standardizing** data refers to transforming each variable so that it more closely follows a **standard** normal distribution, with mean 0 and standard deviation 1.

The [`StandardScaler`](http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.StandardScaler.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01#sklearn.preprocessing.StandardScaler) object in SciKit Learn can do this.


**Generate X and y**:


In [8]:
y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

**Import, fit, and transform using `StandardScaler`**


In [9]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
X_ss = s.fit_transform(X)

### Exercise:

Confirm standard scaling


In [10]:
X2 = np.array(X)
man_transform = (X2-X2.mean(axis=0))/X2.std(axis=0)
np.allclose(man_transform, X_ss)

a = np.array([[1, 2, 3], 
              [4, 5, 6]]) 
print(a) # 2 rows, 3 columns

[[1 2 3]
 [4 5 6]]


In [11]:
a.mean(axis=0) # mean along the *columns*

array([2.5, 3.5, 4.5])

In [12]:
a.mean(axis=1) # mean along the *rows*

array([2., 5.])

### Coefficients with and without scaling


In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
lr = LinearRegression()

y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

In [15]:
lr.fit(X, y)
print(lr.coef_) # min = -18

[ -0.002  -0.122   0.049   0.013   2.858 -18.547   3.686   0.001  -1.508
   0.307  -0.012  -0.933  -0.554]


#### Discussion (together):

The coefficients are on widely different scales. Is this "bad"?


In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
s = StandardScaler()
X_ss = s.fit_transform(X)

In [18]:
lr2 = LinearRegression()
lr2.fit(X_ss, y)
print(lr2.coef_) # coefficients now "on the same scale"

[-0.354 -1.05   1.13   0.088  0.725 -2.147  2.587  0.031 -3.172  2.674
 -2.016 -2.018 -3.949]


### Exercise:

Based on these results, what is the most "impactful" feature (this is intended to be slightly ambiguous)? "In what direction" does it affect "y"?

**Hint:** Recall from last week that we can "zip up" the names of the features of a DataFrame `df` with a model `model` fitted on that DataFrame using:

```python
dict(zip(df.columns.values, model.coef_))
```


In [19]:
pd.DataFrame(zip(X.columns, lr2.coef_)).sort_values(by=1)

,0,1
12,LSTAT,-3.948774
8,DIS,-3.171976
5,NOX,-2.147002
11,PTRATIO,-2.017656
10,TAX,-2.016350
1,CRIM,-1.049828
0,Unnamed: 0,-0.354417
7,AGE,0.030866
3,INDUS,0.087954
4,CHAS,0.725320


Looking just at the strength of the standardized coefficients LSTAT, DIS, RM and RAD are all the 'most impactful'. Sklearn does not have built in statistical signifigance of each of these variables which would aid in making this claim stronger/weaker


### Lasso with and without scaling


We discussed Lasso in lecture.

Let's review together:

1.  What is different about Lasso vs. regular Linear Regression?
2.  Is standardization more or less important with Lasso vs. Linear Regression? Why?


In [20]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

#### Create polynomial features


[`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [21]:
pf = PolynomialFeatures(degree=2, include_bias=False,)
X_pf = pf.fit_transform(X)

**Note:** We use `include_bias=False` since `Lasso` includes a bias by default.


In [22]:
X_pf_ss = s.fit_transform(X_pf)

### Lasso


[`Lasso` documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [23]:
las = Lasso()
las.fit(X_pf_ss, y)
las.coef_ 

array([-0.   , -0.   ,  0.   , -0.   ,  0.   , -0.   ,  0.   , -0.   ,
       -0.   , -0.   , -0.   , -0.984, -0.   , -0.   , -0.   ,  0.   ,
       -0.   ,  0.047, -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.   ,  0.   , -0.   ,  0.034, -0.   , -0.   ,
       -0.   , -0.105, -0.   , -0.   , -0.   , -0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   , -0.   ,  0.   , -0.   , -0.   , -0.   , -0.064, -0.   ,
       -0.   , -0.   , -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.   , -0.   ,  3.425, -0.   , -0.   , -0.   ,
       -0.   , -0.   , -3.621, -0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.   , -0.152, -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ])

### Exercise

Compare

*   Sum of magnitudes of the coefficients
*   Number of coefficients that are zero

for Lasso with alpha 0.1 vs. 1.

Before doing the exercise, answer the following questions in one sentence each:

*   Which do you expect to have greater magnitude?
*   Which do you expect to have more zeros?


In [24]:
las01 = Lasso(alpha = 0.1)
las01.fit(X_pf_ss, y)
print('sum of coefficients:', abs(las01.coef_).sum() )
print('number of coefficients not equal to 0:', (las01.coef_!=0).sum())

sum of coefficients: 26.9026090353222
number of coefficients not equal to 0: 21


In [25]:
las1 = Lasso(alpha = 1)
las1.fit(X_pf_ss, y)
print('sum of coefficients:',abs(las1.coef_).sum() )
print('number of coefficients not equal to 0:',(las1.coef_!=0).sum())

sum of coefficients: 8.431668334789872
number of coefficients not equal to 0: 8


With more regularization (higher alpha) we will expect the penalty for higher weights to be greater and thus the coefficients to be pushed down. Thus a higher alpha means lower magnitude with more coefficients pushed down to 0.


### Exercise: $R^2$


Calculate the $R^2$ of each model without train/test split.

Recall that we import $R^2$ using:

```python
from sklearn.metrics import r2_score
```


In [26]:
from sklearn.metrics import r2_score
r2_score(y,las.predict(X_pf_ss))

0.7185629659673797

#### Discuss:

Will regularization ever increase model performance if we evaluate on the same dataset that we trained on?


## With train/test split


#### Discuss

Are there any issues with what we've done so far?

**Hint:** Think about the way we have done feature scaling.

Discuss in groups of two or three.


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

In [29]:
X_train_s = s.fit_transform(X_train)
las.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las.predict(X_test_s)
r2_score(y_test, y_pred)

0.6758003858672721

In [30]:
X_train_s = s.fit_transform(X_train)
las01.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las01.predict(X_test_s)
r2_score(y_test, y_pred)

0.7973797959308028

### Exercise

#### Part 1:

Do the same thing with Lasso of:

*   `alpha` of 0.001
*   Increase `max_iter` to 100000 to ensure convergence.

Calculate the $R^2$ of the model.

Feel free to copy-paste code from above, but write a one sentence comment above each line of code explaining why you're doing what you're doing.

#### Part 2:

Do the same procedure as before, but with Linear Regression.

Calculate the $R^2$ of this model.

#### Part 3:

Compare the sums of the absolute values of the coefficients for both models, as well as the number of coefficients that are zero. Based on these measures, which model is a "simpler" description of the relationship between the features and the target?


In [31]:
# Part 1

# Decreasing regularization and ensuring convergence
las001 = Lasso(alpha = 0.001, max_iter=100000)

# Transforming training set to get standardized units
X_train_s = s.fit_transform(X_train)

# Fitting model to training set
las001.fit(X_train_s, y_train)

# Transforming test set using the parameters defined from training set
X_test_s = s.transform(X_test)

# Finding prediction on test set
y_pred = las001.predict(X_test_s)

# Calculating r2 score
print("r2 score for alpha = 0.001:", r2_score(y_test, y_pred))


# Part 2

# Using vanilla Linear Regression
lr = LinearRegression()

# Fitting model to training set
lr.fit(X_train_s, y_train)

# predicting on test set
y_pred_lr = lr.predict(X_test_s)

# Calculating r2 score
print("r2 score for Linear Regression:", r2_score(y_pred_lr, y_test))


# Part 3
print('Magnitude of Lasso coefficients:', abs(las001.coef_).sum())
print('Number of coeffients not equal to 0 for Lasso:', (las001.coef_!=0).sum())

print('Magnitude of Linear Regression coefficients:', abs(lr.coef_).sum())
print('Number of coeffients not equal to 0 for Linear Regression:', (lr.coef_!=0).sum())

r2 score for alpha = 0.001: 0.9002771635811371
r2 score for Linear Regression: 0.8667029116056796
Magnitude of Lasso coefficients: 405.7903104487555
Number of coeffients not equal to 0 for Lasso: 87
Magnitude of Linear Regression coefficients: 1315.8819091566234
Number of coeffients not equal to 0 for Linear Regression: 104


## L1 vs. L2 Regularization


As mentioned in the deck: `Lasso` and `Ridge` regression have the same syntax in SciKit Learn.

Now we're going to compare the results from Ridge vs. Lasso regression:


[`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [32]:
from sklearn.linear_model import Ridge

### Exercise

Following the Ridge documentation from above:

1.  Define a Ridge object `r` with the same `alpha` as `las001`.
2.  Fit that object on `X` and `y` and print out the resulting coefficients.


In [33]:
# Decreasing regularization and ensuring convergence
r = Ridge(alpha = 0.001)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_r = r.predict(X_test_s)

# Calculating r2 score
r.coef_

array([ 10.98 , -36.022,   6.93 , -31.814,   5.311, -10.658,  11.214,
        20.566, -33.511,  54.4  ,  -7.696,  11.281,  -0.323,  -1.111,
        22.528,  -0.096,   1.791,  -2.729,   0.231,   3.937,  -1.283,
        -3.323,  -6.845,  -4.655, -11.096,   4.371,   0.853,  -0.042,
         4.45 ,   1.446,  -6.589,  12.3  ,  -1.819,  -5.59 , -16.903,
       -10.606,  30.65 ,   5.532,  -0.07 ,   0.431,   0.529,  -6.469,
         0.989,   1.648,   0.27 ,  -1.66 ,   4.008,  -3.814,  -1.449,
         8.448,  -0.333,   3.262,  13.514,   6.211,   1.928,  18.401,
        -1.557,   1.141,  -4.905,   5.311,  -3.39 ,  -8.142,   1.551,
         0.213,   1.492,  -2.345,   2.293,  -0.847,   8.117,   0.002,
        -9.792,  20.016, -23.329,  32.028, -19.075,  11.548,   4.878,
       -10.615,  -3.199,  -8.951, -18.114,  -4.978, -10.785,   0.474,
        -1.188,  12.775,  -5.947,  -5.052,  -3.62 ,  12.83 ,   0.035,
        -0.095,  -3.871,   1.797, -40.22 ,  55.528, -45.952,  -8.358,
        -7.369,  11.

In [34]:
las001 # same alpha as Ridge above

Lasso(alpha=0.001, max_iter=100000)

In [35]:
las001.coef_

array([  2.416,  -1.462,  -0.   , -18.422,   6.919,  -0.   ,  11.707,
         7.569, -21.85 ,   6.47 ,   0.   ,   2.512,   1.157,  -0.647,
        12.231,   0.   ,   0.967,  -0.   ,  -3.575,   3.873,  -1.374,
        -2.163,  -1.691,  -3.036,  -2.014,   5.025,   0.987,  -0.23 ,
        -0.   ,   1.73 , -10.122,   7.729,  -2.082,  -3.961,  -9.298,
        -0.   ,  -0.   ,   4.677,   0.352,   0.344,  -0.   ,  -1.859,
         0.   ,   1.284,  -0.   ,  -0.84 ,   2.298,  -0.093,  -1.3  ,
         3.271,  -1.17 ,   6.227,   8.969,   4.521,   1.029,   6.   ,
         2.007,  -2.733,  -4.727,   0.794,  -2.473,  -7.764,   2.111,
         1.118,  -0.746,  -0.   ,   0.345,  -0.921,   0.   ,  -1.031,
        -2.217,   8.231,  -3.001,   0.   ,  -5.641,   6.642,   3.86 ,
        -7.311,  -0.   ,  -6.976, -14.831,  -5.817,  -9.071,  -0.618,
        -0.217,   8.659,  -1.697,  -1.148,  -2.227,   8.635,   0.221,
         0.641,  -0.985,   1.658, -19.873,  22.805,  -0.   ,  -7.027,
        -0.   ,  15.

In [36]:
print(np.sum(np.abs(r.coef_)))
print(np.sum(np.abs(las001.coef_)))

print(np.sum(r.coef_ != 0))
print(np.sum(las001.coef_ != 0))

928.163096605384
405.7903104487555
104
87


**Conclusion:** Ridge does not make any coefficients 0. In addition, on this particular dataset, Lasso provides stronger overall regularization than Ridge for this value of `alpha` (not necessarily true in general).


In [ ]:
y_pred = r.predict(X_pf_ss)
print(r2_score(y, y_pred))

y_pred = las001.predict(X_pf_ss)
print(r2_score(y, y_pred))

**Conclusion**: Ignoring issues of overfitting, Ridge does slightly better than Lasso when `alpha` is set to 0.001 for each (not necessarily true in general).


# Example: Does it matter when you scale?


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.3, 
                                                    random_state=72018)

In [38]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

0.6891282353293848

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=72018)

In [40]:
s = StandardScaler()
lr_s = LinearRegression()
X_train_s = s.fit_transform(X_train)
lr_s.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_s = lr_s.predict(X_test_s)
r2_score(y_test, y_pred)

0.6891282353293848

**Conclusion:** It doesn't matter whether you scale before or afterwards, in terms of the raw predictions, for Linear Regression. However, it matters for other algorithms. Plus, as we'll see later, we can make scaling part of a `Pipeline`.


***

### Machine Learning Foundation (C) 2020 IBM Corporation
